In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import os

from cosmo_hydro_emu.pca import *
from cosmo_hydro_emu.viz import *
from cosmo_hydro_emu.load_hacc import *
from cosmo_hydro_emu.emu import *
from cosmo_hydro_emu.gp import *

# Config

In [ ]:
DirIn = '../data/400MPC_RUNS_5SG_2COSMO_PARAM/HAvoCC/'

start_sim_idx = 1
num_sims = 39
exp_variance = 0.95

test_indices = [3, 14, 35]

PARAM_NAME = ['$\\kappa_\\text{w}$',
              '$e_\\text{w}$',
              '$M_\\text{seed}/10^{6}$',
              '$v_\\text{kin}/10^{4}$',
              '$\\epsilon_\\text{kin}/10^{1}$',
              '$\\omega_\\text{m}$',
              '$\\sigma_\\text{8}$'
              ]

# Load parameters

In [ ]:
def read_params(fileIn):
    params_all = np.loadtxt(fileIn, delimiter=",", skiprows=1)
    start_sim_idx = 1
    end_sim_idx = 40
    print('Loaded file with params: ', params_all.shape)
    return params_all[start_sim_idx:end_sim_idx]

fileIn = '/home/nramachandra/Projects/Hydro_runs/CosmoHydro/data/FinalDesign.txt'
params32 = read_params(fileIn)

params32[:, 2] = params32[:, 2] / seed_mass_scale
params32[:, 3] = params32[:, 3] / vkin_scale
params32[:, 4] = params32[:, 4] / eps_scale

print(params32.shape)

# Scatter matrix -- experimental design

In [ ]:
df_train_a = pd.DataFrame(params32, columns=PARAM_NAME)
colors = ['b'] * params32.shape[0]
plot_scatter_matrix(df_train_a, colors);
plt.savefig('../plots/exp_design_csfr.png', bbox_inches='tight')

# Load CSFR data

In [ ]:
scale_factor, csfr_arr = read_csfr(DirIn, num_sims, start_sim_idx=start_sim_idx)
print('scale_factor shape:', scale_factor.shape)
print('csfr_arr shape:', csfr_arr.shape)

# Visualize CSFR

In [ ]:
color_by_index = 4

plt_strings = plot_strings('CSFR')
mlim1, mlim2 = mass_conds('CSFR')

f, a = plt.subplots(1, 1, figsize=(6, 3))

f = plot_lines_with_param_color(params32[:, color_by_index],
                                scale_factor,
                                csfr_arr,
                                *plt_strings,
                                PARAM_NAME[color_by_index],
                                mlim1,
                                mlim2,
                                a);

plt.title('Cosmic Star Formation Rate')
plt.savefig('../plots/CSFR_design.png', bbox_inches='tight')

# Data prep + GP training

In [ ]:
# NaN interpolation
csfr_arr_extra = fill_nan_with_interpolation(csfr_arr, 'linear')

plt.plot(csfr_arr.T, 'k', alpha=0.5);
plt.plot(csfr_arr_extra.T, 'r-.', alpha=0.5);

In [ ]:
## Data prep
z_index = 0

plt_strings = plot_strings('CSFR')
mlim1, mlim2 = mass_conds('CSFR')
mass_cond = np.where((scale_factor >= mlim1) & (scale_factor <= mlim2))

p_all = params32
y_vals = csfr_arr_extra[:, mass_cond][:, 0, :]
y_ind = scale_factor[mass_cond]

print('y_vals range:', y_vals.min(), y_vals.max())

#################################################################

# Train-test split
input_params = p_all[test_indices]
target_vals = y_vals[test_indices]

train_indices = [i for i in np.arange(num_sims) if i not in test_indices]
p_all_train = p_all[train_indices]
y_vals_train = y_vals[train_indices]

#################################################################

## Fitting
sepia_data = sepia_data_format(p_all_train, y_vals_train, y_ind)
print(sepia_data)
model_filename = '../models/CSFR_multivariate_model_z_index' + str(z_index)

sepia_model = do_pca(sepia_data, exp_variance=exp_variance)
sepia_model = do_gp_train(sepia_model, model_filename)
sepia_model = gp_load(sepia_model, model_filename)
plot_train_diagnostics(sepia_model)

# PCA basis

In [ ]:
from sepia import SepiaPlot
SepiaPlot.plot_K_basis(sepia_data, max_plots=1);

# Validation

In [ ]:
pred_mean, pred_quant = emulate(sepia_model, input_params)
plt_strings = plot_strings('CSFR')
mlim1, mlim2 = mass_conds('CSFR')

validation_plot(y_ind, target_vals, pred_mean, pred_quant, *plt_strings, mlim1, mlim2, 'linear', 'linear');
plt.savefig('../plots/CSFR_valid.png', bbox_inches='tight')

# Sensitivity

In [ ]:
f = sensitivity_plot(y_ind, p_all, sepia_model, emulate, PARAM_NAME, *plt_strings, mlim1, mlim2, 'linear', 'linear')
plt.savefig('../plots/CSFR_sensi.png', bbox_inches='tight')